In [1]:
import matplotlib.pyplot as plt
import numpy as np 
from math import sqrt 
from math import floor
from math import ceil
from math import exp 
import pickle
from operator import itemgetter
import time
from datetime import timedelta
import matplotlib
from matplotlib import cm, colors
from numpy import amin, amax, ravel
import pandas as pd 
import pickle

In [2]:
def get_all_saws(current_paths, length, current_conformation):
    """ Все созданные конформации лежат в первом аргументе функции get_all_saws, 
    поэтому первым аргументом нужно подавать переменную, которой ранее присвоили пустой список
    
    """
    if(length==1):
        current_paths.append(current_conformation)
    else:
        for step in [(1, 0), (-1, 0), (0, 1),  (0, -1)]:
            new_point = (current_conformation[-1][0]+step[0], current_conformation[-1][1]+step[1] )
            if new_point in current_conformation:
                continue
            else:
                temp_path = current_conformation.copy()
                temp_path.append(new_point)
                get_all_saws(current_paths, length-1,  temp_path)

In [3]:
def end_to_end_distance(path):
    """Евклидова метрика"""
    distance = (path[-1][0]-path[0][0])**2+(path[-1][1]-path[0][1])**2   
    return distance 

In [4]:
steps = [(1, 0), (-1, 0), (0, 1),  (0, -1)]
def xy_energy(saw, seq):
    """Функция принимает на вход конформацию на плоскости(список координат). 
    Возвращает число топологических соседей для нее """
    k=0.0
    for i in range(1, len(saw)-1):
        for step in steps:
            new_point = (saw[i][0]+step[0], saw[i][1]+step[1] )
            if (new_point in saw    ):
                position = saw.index(new_point)
                k=k+np.cos(seq[i]-seq[position])
                
    for step in steps:
        new_point_begin = (saw[0][0]+step[0], saw[0][1]+step[1] )
        new_point_end= ( saw[-1][0]+step[0], saw[-1][1]+step[1])
        
        if(new_point_begin in saw ):
            position = saw.index(new_point_begin)
            k=k+np.cos(seq[0]-seq[position])
 
        if( new_point_end in saw    ):
            position = saw.index(new_point_end)
            k=k+np.cos(seq[-1]-seq[position])
        
        
    #print(k//2)
    return k/2.0

In [5]:
h = 0
def calc_r (eps, f = 8):

    mean_c_l = []
    
    for i in range(4, f+1):
        
        conformat = []

        start_conformation = [(0,0)]
        get_all_saws(conformat, i ,start_conformation)

        Z = 0.0
        R_sum = 0.0
        
        seq = np.random.uniform(-np.pi,np.pi, (10000, i))
         
        for l in range(len(conformat)):
            r = end_to_end_distance(conformat[l])
            for j in range(len(seq)):
                E = xy_energy(conformat[l], seq[j])
                Z = Z + np.exp(-(-eps*E ))
                 
                R_sum = R_sum + np.exp(-(-eps*E ))*r

        mean_c_l.append(R_sum/Z)
        
 
            
        print(i)
        #print(R_sum,Z)
        print(R_sum/Z)

In [7]:
h = 0
def calc_checks(eps, f = 8, conformat= None):
    if (conformat==None):
        
        conformat = []

        start_conformation = [(0,0)]
        get_all_saws(conformat, i ,start_conformation)

    Z = 0.0
    R_sum = 0.0

    seq = np.random.uniform(-np.pi,np.pi, (600, f))

    sin = 0.0
    cos = 0.0 
    mag2 = 0.0
    e_=0
    for l in range(len(conformat)):
        r = end_to_end_distance(conformat[l])
        for j in range(len(seq)):
            E = xy_energy(conformat[l], seq[j])
            sin_ = np.mean(np.sin(seq[j]))
            cos_ = np.mean(np.cos(seq[j]))
            mag2_ = np.mean(np.cos(seq[j]))**2+np.mean(np.sin(seq[j]))**2
            Z = Z + np.exp((-eps*E ))

            R_sum = R_sum + np.exp((-eps*E ))*r
            sin+= (np.exp((-eps*E ))*sin_)
            cos+= (np.exp((-eps*E ))*cos_)
            mag2+= (np.exp((-eps*E ))*mag2_)
            e_+= (np.exp((-eps*E ))*(-E))
          
    #print("E", e_/Z)
    #print("mag2 ", mag2/Z)
    
    return R_sum/Z,e_/Z,mag2/Z

 
    #print(R_sum,Z)
    #print(R_sum/Z)
    #print("sin ", sin/Z)
    #print("cos ", cos/Z)
    #print("mag2 ", mag2/Z)
    #print("E", e_/Z)
 

In [8]:
N = 10
L = [8]
J = [0, 0.3, 0.6, 0.9, 1.2, 1.5]
 

results = []
for l in L:
    conformat = []
    start_conformation = [(0,0)]
    get_all_saws(conformat, l ,start_conformation)
    for j in J:
        Rs = np.zeros(N)
        Rerss = np.zeros(N)
        Es = np.zeros(N)
        Eerss = np.zeros(N)      
        M2s = np.zeros(N)
        M2erss = np.zeros(N)
        for i in range(N):
            Rs[i], Es[i], M2s[i] = calc_checks(-j, l, conformat)
            print(i, end = " ")
        results.append(
        [l, j, 0, np.mean(Rs), np.std(Rs)
        , np.mean(Es), np.std(Es)
        , np.mean(M2s), np.std(M2s)])
        print(l, j)

results = np.array(results)
#results          
np.savetxt('shortresults1.txt',results)        

0 1 2 3 4 5 6 7 8 9 8 0
0 1 2 3 4 5 6 7 8 9 8 0.3
0 1 2 3 4 5 6 7 8 9 8 0.6
0 1 2 3 4 5 6 7 8 9 8 0.9
0 1 2 3 4 5 6 7 8 9 8 1.2
0 1 2 3 4 5 6 7 8 9 8 1.5


In [10]:
results = np.array(results)
            
np.savetxt('EE/shortresults.txt',results)        

In [23]:
conformat = []

start_conformation = [(0,0)]
get_all_saws(conformat, 6 ,start_conformation)

In [10]:
calc_checks(-0.8, 6, conformat)

9.030904168393608
sin  -0.003096845875214851
cos  0.048870342601087045
mag2  0.13190733196570564
E -1.3676966850163097


In [22]:
calc_checks(1.4, 5, conformat)

6.209641703358412
sin  0.08888718579155522
cos  0.01732769609168026
mag2  0.5282857262894496
E 2.5145492078246585


In [17]:
calc_checks(0, 5, conformat)

7.04
sin  -0.0060536687379127855
cos  -0.00916836332058467
mag2  0.19721833271478106
E -0.03414450728842748


In [15]:
data = pd.read_csv("CPP/XY_0.800000_0.000000_6_0.txt", sep = " ")
data.iloc[0]

N                    6.000000e+00
J                    8.000000e-01
h                    0.000000e+00
mean_R_sq            9.014120e+00
err_mean_R_sq        1.759640e-02
mean_R_gyr_sq        1.474150e+00
err_mean_R_gyr_sq    1.433180e-03
mean_e               3.664320e-01
err_mean_e           9.675000e-04
mean_e_sq            2.082390e-01
err_mean_e_sq        7.673040e-04
mean_e_fourth        9.041950e-02
err_mean_e_fourth    6.240110e-04
mean_sin             7.689870e-03
err_mean_sin         1.797560e-03
mean_cos             2.609470e-02
err_mean_cos         2.245110e-03
mean_m_sq            3.493790e-01
err_mean_m_sq        8.588910e-04
mean_m_fourth        1.816520e-01
err_mean_m_fourth    6.950630e-04
steps                1.000000e+07
bulk2                6.363660e-01
err_bulk2            3.288610e-04
bulk3                1.206460e-01
err_bulk3            4.526100e-04
bulk4                0.000000e+00
err_bulk4            0.000000e+00
lambda1              1.254180e+00
err_lambda1   

In [21]:
data["mean_m_sq"]

0    0.530089
Name: mean_m_sq, dtype: float64